# Python Project Code Review and Grading

### Project Review: Strengths and Improvements

#### Strengths

1. **Separation of Concerns**: - Each module addresses a specific task, demonstrating a clear separation of concerns. This modularity makes the codebase more readable and maintainable, as each module can be modified independently without affecting the others.

2. **SQLAlchemy Core Usage for Database Operations**:  - SQLAlchemy Core is effectively used across the project, allowing for structured, secure database operations. For instance, `stock_details.py` uses SQLAlchemy Core to prepare and execute queries securely, minimizing the risk of SQL injection.

3. **Understanding of `staticmethod` in OOP**:  - The use of `staticmethod` in `stock_details.py` for `validate_inputs` is an appropriate design choice. Since the method only validates inputs and doesn’t rely on any instance or class properties, defining it as a `staticmethod` prevents unnecessary instance creation and reflects a solid grasp of OOP.

4. **Logging Mechanism**:  - The custom logging mechanism in `demo_logging.py` allows for both file and database logging, which is crucial for tracking errors and system events. While there are some issues with its current design (discussed below), the intent to log key events is well implemented and provides robust traceability for debugging and audits in production systems.

#### Areas for Improvement

1. **Module Consolidation and Redundant Modules**:
- **Issue**: Some modules, such as `python_project.py` and `python_project_etl.py`, have redundant code or responsibilities.
 
- **Improvement**: Consolidate smaller, redundant modules into larger, reusable ones (e.g., `utils.py` or `db_setup.py`) to simplify the codebase and improve maintainability.

2. **Configuration and Sensitive Data Management**:  

- **Issue**: Database credentials, file paths, and other configurations are hardcoded within the files, which is a security risk and hinders collaboration.

- **Improvement**: Store sensitive data like database credentials in a `.env` file and provide a sample `.env.example` for other collaborators to set up their environment securely.

3. **Logging and Error Handling in `python_project_etl.py`**:
- **Issue**: The `python_project_etl.py` module sets up logging with SQLAlchemy ORM, which introduces unnecessary complexity and overhead, and logs are managed with `print()` statements in `stocks_stats.py`.

- **Improvement**: Move logging to a shared module using SQLAlchemy Core or `mysql.connector` for efficiency and consistency, and replace `print()` statements with logging to provide structured, searchable logs across all modules.

4. **Use of CTEs in `stocks_stats.py`**:
  
- **Issue**: The multiple nested CTEs add unnecessary complexity and may reduce query performance. Calculating basic statistics can be achieved more efficiently using aggregate functions and basic arithmetic without over-relying on CTEs.

- **Improvement**: Simplify the logic by querying directly with aggregates, reducing execution time and complexity.

In [ ]:
from sqlalchemy import select, func, and_

# Define table metadata for the stock_rates table
stock_rates = Table('stock_rates', metadata, autoload_with=engine)

# Simplified query with yield calculation
query = (
    select(
        stock_rates.c.Symbol,
        func.max(stock_rates.c.High).label("MaxRate"),
        func.min(stock_rates.c.Low).label("MinRate"),
        func.avg(stock_rates.c.Close).label("AvgRate"),
        ((func.max(stock_rates.c.Close) - func.min(stock_rates.c.Close)) / func.min(stock_rates.c.Close) * 100).label("Yield")
    )
    .where(and_(stock_rates.c.Date >= start_date, stock_rates.c.Date <= end_date))
    .group_by(stock_rates.c.Symbol)
)

# Execute query
result = engine.execute(query)
for row in result:
    print(row)

5. **Empty Exception Class**:  

- **Issue**: The `InvalidInputError` exception class in `stock_details.py` currently only contains a `pass` statement, making it functionally identical to a general `Exception`. This provides no additional debugging information or specialized handling.

- **Improvement**: Enhance the `InvalidInputError` by logging parsed error details to a database when the exception is raised. This will allow for easier tracking of invalid input patterns.

In [ ]:
from logger_setup import logger  # Assuming logger is set up in logger_setup
from sqlalchemy.exc import SQLAlchemyError

# Custom InvalidInputError with logging functionality
class InvalidInputError(Exception):
    def __init__(self, message="Invalid input provided", details=None):
        super().__init__(message)
        self.details = details
        self.log_error()

    def log_error(self):
        """Log error details to the database using an external logger"""
        try:
            # Log structured error message with context details
            logger.error(
                f"InvalidInputError: {self.details.get('error_msg', 'Invalid input encountered')}",
                extra={"context": self.details.get("context", "N/A")}
            )
        except SQLAlchemyError as e:
            logger.error(f"Logging failed: {e}")

#### Usage in `validate_inputs` Method

In [ ]:
@staticmethod
def validate_inputs(symbol: str, start_date: str, end_date: str):
    if not symbol or not isinstance(symbol, str):
        raise InvalidInputError(details={"error_msg": "Symbol is invalid", "context": f"Symbol: {symbol}"})
    # Additional validation logic...

6. **Generic Exception Handling in `stocks_stats.py`**:

- **Issue**: The exception handling in `stocks_stats.py` uses `except Exception as e`, which is overly generic. This does not add value to the debugging process, and printing tracebacks directly isn’t best practice.

- **Improvement**: Implement specific exception handling, such as `SQLAlchemyError` or `ValueError`, to provide clearer error feedback. Avoid printing full tracebacks and instead log errors for a structured record.

7. **No Upsert Logic in ETL Process**:

- **Issue**: The `python_project_etl.py` file lacks upsert logic, so duplicate data may be inserted during the ETL process.

- **Improvement**: Implement an upsert mechanism to avoid duplicates.

1) **Example (Custom SQLAlchemy on_duplicate_key_update Method - [*StackOverflow*](https://stackoverflow.com/questions/42461959/how-do-i-perform-an-update-of-existing-rows-of-a-db-table-using-a-pandas-datafra))**:
```python
    def create_method(meta):
        def method(table, conn, keys, data_iter):
            sql_table = db.Table(table.name, meta, autoload=True)
            insert_stmt = db.dialects.mysql.insert(sql_table).values([dict(zip(keys, data)) for data in data_iter])
            upsert_stmt = insert_stmt.on_duplicate_key_update({x.name: x for x in insert_stmt.inserted})
            conn.execute(upsert_stmt)

        return method

    engine = db.create_engine(...)
    conn = engine.connect()
    with conn.begin():
        meta = db.MetaData(conn)
        method = create_method(meta)
        df.to_sql(table_name, conn, if_exists='append', method=method)
```
2) **You can consider Using a custom lib like - [*pandas-upsert-to-mysql*](https://pypi.org/project/pandas-upsert-to-mysql/)**

3) **MySQL `ON DUPLICATE KEY UPDATE`**:
```sql
INSERT INTO stock_history (symbol, date, open, close)
VALUES (:symbol, :date, :open, :close)
ON DUPLICATE KEY UPDATE
  open = VALUES(open),
  close = VALUES(close);
```

- **Conclusion**: The ETL process works well, but adding upsert functionality would handle duplicate records more effectively. No points deducted for this step.

8. **Lack of `main.py` to Coordinate Modules**

- **Issue**: Currently, there is no main module that coordinates the operations across all modules, which makes it harder to execute end-to-end tasks from a single entry point.

- **Improvement**: Create a `main.py` file that initializes configuration, sets up logging, and calls the various functions and classes for stock details, statistics, and ETL processes. This provides a single entry point, improving usability.

In [ ]:
from stock_details import get_stock_details
from stocks_stats import get_stocks_stats
from python_project_etl import run_etl_process

def main():
    # Set up logging and configuration

    # Run ETL process
    run_etl_process()

    # Fetch stock details
    get_stock_details('MSFT', '2023-01-01', '2023-02-01')

    # Fetch stock statistics
    get_stocks_stats(['AAPL', 'MSFT'], '2023-01-01', '2023-02-01')

if __name__ == "__main__":
    main()

### Grading Table
| Area of Improvement                            | Points Deducted |
|------------------------------------------------|-----------------|
| Module Consolidation and Redundant Code        | 2               |
| Hardcoded Configurations and Sensitive Data    | 2               |
| Unnecessary Complexity in Logging              | 2               |
| Over-Engineered CTE Logic making stock_stats.py code more complex and long                    | 2               |
| Generic Exception Handling (in 1 module) and Empty Exception Class | 2               |

**Total Points Deducted: 10**

### Final Grade: 90/100